In [31]:
# import statements

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pickle
from math import comb
import statistics

In [3]:
ibs_graph = pickle.load(open("ibs_graph.pkl", "rb"))

In [52]:
# CDF (cumulative distribution function)
# calculate Pr(v <= x): probability a node v has <= x degree
# MARKOVS INEQUALITY - MAKE SURE WE INCLUDE IN PAPER

n = len(ibs_graph.nodes)
expected = n / 2

degree_dict = {}

cdf = np.zeros(100)
max_deg = 0

for node in ibs_graph.nodes:
    degree = len(ibs_graph.edges(node))
    cdf[degree] += 1
    if degree > max_deg:
        max_deg = degree

cdf_sum = np.zeros(max_deg + 1)
markovs = np.zeros(max_deg + 1)

for i in range(len(cdf_sum)):
    if i >= 1:
        cdf_sum[i] = cdf[i] + cdf_sum[i-1]

    else:
        cdf_sum[i] = cdf[i]
    if i > 0:
        markovs[i] = 1 - expected / i
    else: 
        markovs[i] = 0
    markovs[i] = max(0, markovs[i])
    markovs[i] = min(1, markovs[i])

degrees = np.arange(0, max_deg + 1, 1)

cdf_sum = cdf_sum / n

# print(degrees)
# print(cdf_sum)
print(markovs)


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


In [51]:
# plot

plt.figure(figsize = (15, 15), dpi = 600)
plt.plot(degrees, cdf_sum, label = "IBS")
plt.plot(degrees, markovs, label = "Markov's")
plt.xlabel("degree")
plt.ylabel("Cumulative Data Function (CDF), Pr(v <= x)")
plt.legend()

In [44]:
# number of edges

m = len(ibs_graph.edges)
print("vertices:", n, "edges:", m, "average degree:", m / n)

deg_list = []
deg_list_nonzero = []

for node in ibs_graph.nodes:
    deg = len(ibs_graph.edges(node))
    deg_list.append(deg)
    if deg > 0:
        deg_list_nonzero.append(deg)

print("median degree:", statistics.median(deg_list), "median nonzero degree:", statistics.median(deg_list_nonzero))
print("clustering coeff:", nx.average_clustering(ibs_graph))

vertices: 581 edges: 417 average degree: 0.7177280550774526
median degree: 0 median nonzero degree: 5.0
clustering coeff: 0.09301932955578972
